In [ ]:
%run -i ../python/common.py
%run -i ../python/ln_preamble.py

# Introduction
So now that we have a sense for we write code that is very close to the native language of the computer lets try and figure out how ensure that most humans never have to do that!


In [ ]:
print("hello world")